In [5]:
import dash
import dash_mantine_components as dmc
from dash import html, dcc, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px

import webbrowser
import pandas as pd
import numpy as np

from statistics import variance, stdev
from copy import copy

In [6]:
df = pd.read_csv("combined_processed.csv")

In [7]:
castle_heroes = ['Adelaide', 'Orrin', 'Valeska', 'Edric', 'Sylvia', 
                 'Beatrice', 'Lord Haart', 'Sorsha', 'Christian', 
                 'Tyris', 'Rion', 'Adela', 'Cuthbert', 'cuttbert','Ingham', 'Sanya', 'Loynis', 
                 'Caitlin', 'Katarzyna', 'Roland']

rampart_heroes = ['Mephala', 'Ufretin', 'Jenova', 'Ryland', 'Giselle', 'Ivor', 'Clancy', 'Thorgrim', 'Kyrre', 
                  'Coronius', 'Uland', 'Elleshar', 'Gem', 'Malcom', 'Melodia', 'Alagar', 'Aeris']

tower_heroes = ['Piquedram', 'Thane', 'Josephine', 'Neela', 'Torosar', 'Fafner', 'Rissa', 'Iona', 'Astral', 
                'Halon', 'Serena', 'Daremyth','Theodorus', 'Solmyr', 'Cyra', 'Aine', 'Dracon']

inferno_heroes = ['Fiona', 'Rashka', 'Marius', 'Ignatius', 'Octavia', 'Calh', 
                  'Pyre', 'Nymus', 'Ayden', 'Xyron', 'Axsis', 'Olema','Calid', 'Ash', 'Zydar', 'Xarfax']

necropolis_heroes = ['Straker', 'Vokial', 'Moandor', 'Charna','Tamika', 'Isra', 'Clavius', 'Ranloo', 
                     'Septienna', 'Aislinn', 'Sandro', 'Nimbus', 'Thant', 'Xsi', 'Vidomina', 'Nagash',
                    'Haart Lich', 'Galthran']

dungeon_heroes = ['Lorelei', 'lorelai', 'Arlach', 'Dace', 'Ajit', 'Damacon', 'Gunnar', 
                  'Synca', 'Shakti', 'Alamar', 'Jaegar','Malekith', 'Jeddite', 
                  'Deemer', 'Geon', 'Sephinroth', 'Darkstorn', 'Mutare Drake', 'Mutare']

stronghold_heroes = ['Yog', 'Gurnisson', 'Jabarkas', 'Crag Hack', 'Shiva', 'Gretchin', 'Krellion', 
                     'Tyraxor', 'Gird', 'Vey', 'Dessa', 'Terek', 'Zubin', 'Gundula', 'Oris', 'Saurug', 'Boragus']

fortress_heroes = ['Bron', 'Drakon', 'Wystan', 'Tazar', 'Alkin', 'Korbac', 'Gerwulf', 'Broghild', 
                   'Mirlanda', 'Rosic', 'Voy', 'Verdish', 'Kinkeria', 'Merist', 'Styg', 'Andra', 'Tiva']

conflux_heroes = ['Pasis', 'Thunar', 'Ignissa', 'Lacus', 'Kalt', 'Fiur', 'Erdamon', 'Monere', 
                  'Luna', 'Inteus', 'Grindan', 'Labetha', 'Ciele', 'Gelare', 'Aenain', 'Brissa']

cove_heroes = ['Cassiopeia', 'Derek', 'Anabel', 'Illor','Tark', 'Corkes', 'Jeremy', 'Miriam', 'Elmore', 
               'Leena', 'Eovacius','Astra', 'Andal', 'Manfred', 'Casmetra', 'Zilare', 'Spint', 'Dargem']

towns = ["castle", "rampart", "tower", "inferno", "necropolis", "dungeon", "stronghold", "fortress", "conflux", "cove"]
heroes = {"castle": castle_heroes, "rampart": rampart_heroes, "tower": tower_heroes, "inferno": inferno_heroes, 
          "necropolis": necropolis_heroes, "dungeon": dungeon_heroes, "stronghold": stronghold_heroes, 
          "fortress": fortress_heroes, "conflux": conflux_heroes, "cove": cove_heroes}

template_types = ["All", "XL+U", "Mirror", "Jebus", "Duel", "Other"]

In [8]:
if False:
    df2 = pd.DataFrame(columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", "turns", "template"])

    for row in df.iterrows():
        row = row[1]
        if row[1] > row[3]: result = 1
        elif row[1] < row[3]: result = 0
        else: result = 0.5
        df_temp = pd.DataFrame([[result, row[12], row[6], row[13], row[8] , row[14], row[7], row[11], row[5], row[16]]], 
                              columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", 
                                         "turns", "template", "template_type"])
        df2 = pd.concat([df2, df_temp])

        df_temp = pd.DataFrame([[1-result, row[14], row[7], {"red": "blue", "blue": "red", "white": "white"}[row[13]], 
                                 -row[8], row[12], row[6],row[11], row[5], row[16]]], 
                               columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", 
                                          "turns", "template", "template_type"])
        df2 = pd.concat([df2, df_temp])

    df2.to_csv("set2.csv", index = False)

In [9]:
df2 = pd.read_csv("set2.csv")

In [10]:
def town_v_town_winrate_heatmap(int_df):
    res = []
    for town in towns:
        res.append([])
        for op_town in towns[::-1]:
            subset = int_df[(int_df["town"] == town) & (int_df["opponent_town"] == op_town)]
            if (len(subset) < 8) or ((town == op_town) and (len(subset) < 16)):
                winrate = float("nan")
            else:
                winrate = round(sum(subset["result"])/len(subset), 2)
            res[-1].append(winrate)
    
    return res

def town_v_town_bidding_heatmap(int_df):
    res = []
    for town in towns:
        res.append([])
        for op_town in towns[::-1]:
            subset = int_df[(int_df["town"] == town) & (int_df["opponent_town"] == op_town)]
            if (len(subset) < 8) or ((town == op_town) and (len(subset) < 16)):
                bidding = float("nan")
            else:
                bidding = round(sum(subset["bidding"])/len(subset))
            res[-1].append(bidding)
    
    return res

def town_v_town_bidding_variance_heatmap(int_df):
    res = []
    for town in towns:
        res.append([])
        for op_town in towns[::-1]:
            subset = int_df[(int_df["town"] == town) & (int_df["opponent_town"] == op_town)]
            if (len(subset) < 8) or ((town == op_town) and (len(subset) < 16)):
                variance = float("nan")
            else:
                variance = round(stdev(subset["bidding"]))
            res[-1].append(variance)
    
    return res

def create_town_v_town_graphs(int_df):
    res_winrate = town_v_town_winrate_heatmap(int_df)
    res_bidding = town_v_town_bidding_heatmap(int_df)
    res_bidding_variance = town_v_town_bidding_variance_heatmap(int_df)

    fig_winrate = go.Figure(
        data = go.Heatmap(z=res_winrate, x = towns[::-1], y = towns, text=res_winrate, texttemplate="%{text}"),
        layout={"xaxis_title": 'Opponent Town', "yaxis_title":'Player Town', "title": "Town V Town winrate"})
    
    fig_bidding = go.Figure(
        data = go.Heatmap(z=res_bidding, x = towns[::-1], y = towns, text=res_bidding, texttemplate="%{text}"),
        layout={"xaxis_title": 'Opponent Town', "yaxis_title":'Player Town', "title": "Town V Town bidding"})

    fig_bidding_variance = go.Figure(
        data = go.Heatmap(z=res_bidding_variance, x = towns[::-1], y = towns, text=res_bidding_variance, texttemplate="%{text}"),
        layout={"xaxis_title": 'Opponent Town', "yaxis_title":'Player Town', "title": "Town V Town bidding variance"})
    
    return fig_winrate, fig_bidding, fig_bidding_variance

In [11]:
# Town vs Town heatmap (Section two)

used_df = copy(df2)
fig_winrate, fig_bidding, fig_bidding_variance = create_town_v_town_graphs(used_df)

app = dash.Dash(__name__)

app.layout = html.Div([dcc.Dropdown(template_types, value = "All", id="template_dropdown"),
                       dcc.Graph(id="town_V_town_graph"),
                       dcc.Checklist(options = ["bidding", "winrate", "bidding variance"], value = ["bidding"], 
                                     id="town_V_town_check"),
                       dcc.Store(data = [], id="town_V_town_state"),
                       dcc.Store(data = 0, id="dataset")])

@app.callback(
    Output("dataset", "data"),
    Input("dataset", "data"),
    Input("template_dropdown", "value"))
def update_template(cur_data, template):
    global used_df, fig_winrate, fig_bidding, fig_bidding_variance

    if template != "All":
        used_df = df2[df2["template_type"] == template]
    else:
        used_df = copy(df2)

    fig_winrate, fig_bidding, fig_bidding_variance = create_town_v_town_graphs(used_df)
    
    return cur_data+1
    

@app.callback(
    Output("town_V_town_graph", "figure"),
    Output("town_V_town_check", "value"),
    Output("town_V_town_state", "data"),
    Input("town_V_town_check", "value"),
    Input("town_V_town_state", "data"),
    Input("dataset", "data"))
def town_V_town(value, state, cur_data):
    value = list(set(value) - set(state))
    
    if value == ["bidding"] or (value == [] and state == ["bidding"]):
        return fig_bidding, ["bidding"], ["bidding"]
    
    elif value == ["winrate"] or (value == [] and state == ["winrate"]):
        return fig_winrate, ["winrate"], ["winrate"]
    
    elif value == ["bidding variance"] or (value == [] and state == ["bidding variance"]):
        return fig_bidding_variance, ["bidding variance"], ["bidding variance"]

In [12]:
def town_bar_graphs(sub_df):
    bidding_quantiles = []
    turns_quantiles = []
    for i in [0, 0.2, 0.4, 0.6, 0.8, 1]:
        bidding_quantiles.append(sub_df["bidding"].abs().quantile(i))
        turns_quantiles.append(sub_df["turns"].abs().quantile(i))

    for i in range(5):
        sub_df.loc[(sub_df["bidding"] >= bidding_quantiles[i]) & (sub_df["bidding"] > 0) & 
                   (sub_df["bidding"] <= bidding_quantiles[i+1]), "bidding quantiles"] = \
        f"{round(0.2*i,1)}-{round(0.2*(i+1),1)}"

        sub_df.loc[(sub_df["turns"] >= turns_quantiles[i]) & (sub_df["color"] == "red") & 
                   (sub_df["turns"] <= turns_quantiles[i+1]), "turns quantiles"] = \
        f"{round(0.2*i,1)}-{round(0.2*(i+1),1)}"

    sub_df.loc[sub_df["bidding"] <= 0]["bidding quantiles"] = np.nan
    sub_df.loc[sub_df["color"] == "blue"]["turns quantiles"] = np.nan

    bidding_x = []
    turns_x = []
    for i in ["0.0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"]:
        bidding_x.append(round(sub_df[sub_df["bidding quantiles"] == i]["result"].mean(), 2))
        turns_x.append(round(sub_df[sub_df["turns quantiles"] == i]["result"].mean(), 2))
    
    return bidding_x, turns_x

In [13]:
app = dash.Dash(__name__)


app.layout = html.Div([dcc.Graph(id="town_graph"),
                       dcc.Checklist(options = ["bidding", "turns"], value = ["bidding"], 
                                     id="town_check"),
                       dcc.Dropdown(towns, value = towns[0], id='town_dropdown'),
                       dcc.Store(data = [], id="town_state")])

@app.callback(
    Output("town_graph", "figure"),
    Output("town_check", "value"),
    Output("town_state", "data"),
    Input("town_check", "value"),
    Input("town_state", "data"),
    Input("town_dropdown", "value"))
def town_V_town(value, state, town):
    sub_df = df2[df2["town"] == town]
    print(sub_df)

    value = list(set(value) - set(state))
    
    if value == ["bidding"] or (value == [] and state == ["bidding"]):
        bidding_x = town_bar_graphs(sub_df)[0]
        return px.bar(x=["0.0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"], y=bidding_x), ["bidding"], ["bidding"]
    
    elif value == ["turns"] or (value == [] and state == ["turns"]):
        turns_x = town_bar_graphs(sub_df)[1]
        return px.bar(x=["0.0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"], y=turns_x), ["turns"], ["turns"]

In [58]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2023 16:05:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:41] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:41] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:41] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 16:05:46] "POST /_dash-update-component HTTP/1.